In [1]:
import ipywidgets as ipw
import ipydatetime, datetime
from IPython.display import display
import numpy as np
import pandas as pd
from collections import OrderedDict
import json
import aurora.schemas.data_schemas
from aurora.schemas.data_schemas import BatterySpecs, BatteryComposition, BatteryCapacity, BatteryMetadata, BatterySample, BatterySpecsJsonTypes, BatterySampleJsonTypes
from aurora.schemas.convert import pd_dataframe_to_formatted_json, dict_to_formatted_json

# import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib widget

## Load mock data

- TODO: convert all capacities to mAh to make queries simpler?

- We impose the properties data types from this dictionary. They should actually be obtained from the BatterySample pydantic schema... **how do we do that?**

In [2]:
def load_available_specs():
    STD_SPECS = pd.read_csv('sample_specs.csv', dtype=BatterySpecsJsonTypes)
    return STD_SPECS

def load_available_samples():
    # AVAIL_SAMPLES = [BatterySample.parse_obj(dic) for dic in json.load(open('available_samples.json', 'r'))]
    # AVAIL_SAMPLES_D = {battery_id: BatterySample.parse_obj(dic) for battery_id, dic in json.load(open('available_samples_id.json', 'r')).items()}
    with open('available_samples.json', 'r') as f:
        data = json.load(f)
    # load json and enforce data types
    AVAIL_SAMPLES_DF = pd.json_normalize(data)
    AVAIL_SAMPLES_DF = AVAIL_SAMPLES_DF.astype(dtype={col: typ for col, typ in BatterySampleJsonTypes.items() if col in AVAIL_SAMPLES_DF.columns})
    AVAIL_SAMPLES_DF["metadata.creation_datetime"] = pd.to_datetime(AVAIL_SAMPLES_DF["metadata.creation_datetime"])
    return AVAIL_SAMPLES_DF

STD_RECIPIES = {
    'Margherita': ['pomodoro', 'mozzarella'],
    'Capricciosa': ['pomodoro', 'mozzarella', 'funghi', 'prosciutto', 'carciofini', 'olive nere'],
    'Quattro formaggi': ['mozzarella', 'gorgonzola', 'parmigiano', 'fontina'],
    'Vegetariana': ['pomodoro', 'mozzarella', 'funghi', 'zucchine', 'melanzane', 'peperoni']
}

def load_available_recipies():
    global STD_RECIPIES
    return STD_RECIPIES.copy()

In [3]:
available_samples = load_available_samples()
available_specs = load_available_specs()
available_recipies = load_available_recipies()

def update_available_samples():
    global available_samples
    available_samples = load_available_samples()

def update_available_specs():
    global available_specs
    available_specs = load_available_specs()

def update_available_recipies():
    global available_recipies
    available_recipies = load_available_recipies()

In [4]:
def query_available_specs(field: str = None):
    """
    This mock function returns a pandas.DataFrame of allowed specs.
        field (optional): name of a field to query [manufacturer, composition, capacity, form_factor]
    """
    global available_specs
    
    if field:
        return available_specs.get(field).unique().tolist()
    else:
        return available_specs

def query_available_samples(query=None, project=None):
    """
    This mock function returns the available samples.
        query: a pandas query
        project (optional): list of the columns to return (if None, return all)
    
    Returns a pandas.DataFrame or Series
    """
    global available_samples
    
    # perform query
    if query is not None:
        results = available_samples.query(query)
    else:
        results = available_samples
    if project and (results is not None):
        if not isinstance(project, list):
            project = [project]
        return results[project]
    else:
        return results

def query_available_recipies():
    """A mock function that returns the available synthesis recipies."""
    global available_recipies
    return available_recipies

def write_pd_query_from_dict(query_dict):
    """
    Write a pandas query from a dictionary {field: value}.
    Example: 
        write_pandas_query({'manufacturer': 'BIG-MAP', 'battery_id': 98})
    returns "(`manufacturer` == 'BIG-MAP') and (`battery_id` == 98)"
    """
    query = " and ".join(["(`{}` == {})".format(k, f"{v}" if isinstance(v, (int, float)) else f"'{v}'") for k, v in query_dict.items() if v])
    if query:
        return query

## 1. Sample selection

In [5]:
def test_validate_sample_callback_function(obj):
    """Function that validates inputs and returns the needed info."""
    print(obj.selected_sample_dict)
    return obj.selected_sample_dict

def test_validate_specs_recipe_callback_function(obj):
    """Function that validates inputs and returns the needed info."""
    print(obj.selected_specs_dict)
    print(obj.selected_recipe_dict)
    return {'specs': obj.selected_specs_dict, 'recipe': obj.selected_recipe_dict}

### **From Battery ID**

In [6]:
class SampleFromId(ipw.VBox):
    
    BOX_LAYOUT_1 = {'width': '40%'}
    BOX_STYLE = {'description_width': 'initial'}
    BUTTON_STYLE = {'description_width': '30%'}
    BUTTON_LAYOUT = {'margin': '5px'}

    def __init__(self, validate_callback_f):
        
        # initialize widgets
        self.w_id_list = ipw.Select(
            rows=10,
            description="Select Battery ID:",
            style=self.BOX_STYLE, layout=self.BOX_LAYOUT_1)
        self.w_update = ipw.Button(
            description="Update",
            button_style='', tooltip="Update available samples", icon='refresh',
            style=self.BUTTON_STYLE, layout=self.BUTTON_LAYOUT)
        self.w_sample_preview = ipw.Output()
        self.w_validate = ipw.Button(
            description="Validate",
            button_style='success', tooltip="Validate the selected sample", icon='check',
            disabled=True,
            style=self.BUTTON_STYLE, layout=self.BUTTON_LAYOUT)
        
        super().__init__()
        self.children = [
            self.w_id_list,
            self.w_update,
            self.w_sample_preview,
            self.w_validate,
        ]
        
        # initialize options
        if not callable(validate_callback_f):
            raise TypeError("validate_callback_f should be a callable function")
        # self.validate_callback_f = validate_callback_f
        self.w_id_list.value = None
        self.on_update_button_clicked()
        
        # setup automations
        self.w_update.on_click(self.on_update_button_clicked)
        self.w_id_list.observe(handler=self.on_battery_id_change, names='value')
        self.w_validate.on_click(lambda arg: self.on_validate_button_clicked(validate_callback_f))


    @property
    def selected_sample_id(self):
        return self.w_id_list.value
    
    @property
    def selected_sample_dict(self):
        return dict_to_formatted_json(
            query_available_samples(write_pd_query_from_dict({'battery_id': self.w_id_list.value})).iloc[0])

    @staticmethod
    def _build_sample_id_options():
        """Returns a (option_string, battery_id) list."""
        table = query_available_samples(project=['battery_id', 'metadata.name']).sort_values('battery_id')
        return [("", None)] + [(f"<{row['battery_id']:5}>   \"{row['metadata.name']}\"", row['battery_id']) for index, row in table.iterrows()]

    def display_sample_preview(self):
        self.w_sample_preview.clear_output()
        if self.w_id_list.value is not None:
            with self.w_sample_preview:
                display(query_available_samples(write_pd_query_from_dict({'battery_id': self.w_id_list.value})))

    def update_validate_button_state(self):
        self.w_validate.disabled = (self.w_id_list.value is None)

    def on_battery_id_change(self, _ = None):
        self.display_sample_preview()
        self.update_validate_button_state()

    def on_update_button_clicked(self, _ = None):
        update_available_samples()
        self.w_id_list.options = self._build_sample_id_options()

    def on_validate_button_clicked(self, callback_function):
        # call the validation callback function
        return callback_function(self)


w_sample_from_id = SampleFromId(test_validate_sample_callback_function)
w_sample_from_id

SampleFromId(children=(Select(description='Select Battery ID:', layout=Layout(width='40%'), options=(('', None…

### **From Battery Specs** (loaded from available ones)

Function to perform query of samples. Options are taken from available specs (assuming they are representative of the samples' specs).\
Next to each option is the number of samples available. Once I selected the specs, a list of matching samples is generated.

When the user selects a spec value, the options of all the fields must be updated.\
*Be aware that changing the options does not preserve the value, so we must enforce it!*

Each `spec_field` contains the number of structures obtained by filtering the samples *without* the field's current value.

In [7]:
# @ipw.widget.register
# class ComboboxWithOptions(ipw.Combobox):
#     """
#     Single line textbox widget with a dropdown and autocompletion.
#     Supports (label, value) pairs options, like ipw.Select.
#     """
#     def set_options(self, x):
#         print(f'setting options -- value = {self.value}')
#         self._options_full = ipw.widget_selection._make_options(x)
#         self._options_labels = [pair[0] for pair in self._options_full]
#         self._options_values = [pair[1] for pair in self._options_full]
#         self._options_dict = {pair[0]: pair[1] for pair in self._options_full}
#         self.options = self._options_labels
    
#     @property
#     def current_value(self):
#         """The actual value of the chosen option."""
#         return self._options_dict[self.value]
    
#     def __init__(self, **kwargs):
#         super().__init__(**kwargs)
#         self.set_options(self.options)

In [8]:
class SampleFromSpecs(ipw.VBox):

    _DEBUG = False
    QUERIABLE_SPECS = {
        "manufacturer": "Manufacturer",
        "composition.description": "Composition",
        "capacity.nominal": "Nominal capacity",
        "form_factor": "Form factor",
        # "metadata.creation_datetime": "Creation date",
    }
    BOX_LAYOUT_1 = {'width': '40%'}
    BOX_LAYOUT_2 = {'width': '100%', 'height': '100px'}
    BOX_STYLE = {'description_width': '25%'}
    BUTTON_STYLE = {'description_width': '30%'}
    BUTTON_LAYOUT = {'margin': '5px'}
    OUTPUT_LAYOUT = {'max_height': '500px', 'width': '90%', 'overflow': 'scroll', 'border': 'solid 2px', 'margin': '5px', 'padding': '5px'}
    SAMPLE_BOX_LAYOUT = {'width': '90%', 'border': 'solid blue 2px', 'align_content': 'center', 'margin': '5px', 'padding': '5px'}
    
    def __init__(self, validate_callback_f):

        # initialize widgets
        self.w_specs_manufacturer = ipw.Select(
            description="Manufacturer:",
            placeholder="Enter manufacturer",
            layout=self.BOX_LAYOUT_2, style=self.BOX_STYLE)
        self.w_specs_composition = ipw.Select(
            description="Composition:",
            placeholder="Enter composition",
            layout=self.BOX_LAYOUT_2, style=self.BOX_STYLE)
        self.w_specs_capacity = ipw.Select(
            description="Nominal capacity:",
            placeholder="Enter nominal capacity",
            layout=self.BOX_LAYOUT_2, style=self.BOX_STYLE)
        self.w_specs_form_factor = ipw.Select(
            description="Form factor:",
            placeholder="Enter form factor",
            layout=self.BOX_LAYOUT_2, style=self.BOX_STYLE)
        # self.w_specs_metadata_creation_date = ipydatetime.DatetimePicker(
        #     description="Creation time:",
        #     style=BOX_STYLE)
        # self.w_specs_metadata_creation_process = ipw.Text(
        #     description="Creation process",
        #     placeholder="Describe creation process",
        #     style=BOX_STYLE)

        self.w_update = ipw.Button(
            description="Update",
            button_style='', tooltip="Update available samples", icon='refresh',
            style=self.BUTTON_STYLE, layout=self.BUTTON_LAYOUT)
        self.w_reset = ipw.Button(
            description="Reset",
            button_style='danger', tooltip="Clear fields", icon='times',
            style=self.BUTTON_STYLE, layout=self.BUTTON_LAYOUT)
        self.w_query_result = ipw.Output(layout=self.OUTPUT_LAYOUT)

        self.w_select_sample_id = ipw.Dropdown(
            description="Select Battery ID:", value=None,
            layout=self.BOX_LAYOUT_1, style={'description_width': 'initial'})
        self.w_cookit = ipw.Button(
            description="Load/Synthesize new", #['primary', 'success', 'info', 'warning', 'danger', '']
            button_style='primary', tooltip="Synthesize sample with these specs", icon='',
            style=self.BUTTON_STYLE, layout=self.BUTTON_LAYOUT)
        self.w_sample_preview = ipw.Output()
        self.w_validate = ipw.Button(
            description="Validate",
            button_style='success', tooltip="Validate the selected sample", icon='check',
            disabled=True,
            style=self.BUTTON_STYLE, layout=self.BUTTON_LAYOUT)

        super().__init__()
        self.children = [
            ipw.GridBox([
                self.w_specs_manufacturer,
                self.w_specs_composition,
                self.w_specs_capacity,
                self.w_specs_form_factor,
                # self.w_specs_metadata_creation_date,
                # self.w_specs_metadata_creation_process,
            ], layout=ipw.Layout(grid_template_columns="repeat(2, 45%)")),
            ipw.HBox([self.w_update, self.w_reset], layout={'justify_content': 'center', 'margin': '5px'}),
            self.w_query_result,
            ipw.VBox([
                ipw.HBox([self.w_select_sample_id, ipw.Label(' or '), self.w_cookit], layout={'justify_content': 'space-around'}),
                self.w_sample_preview,
            ], layout=self.SAMPLE_BOX_LAYOUT),
            self.w_validate
        ]

        # initialize options
        if not callable(validate_callback_f):
            raise TypeError("validate_callback_f should be a callable function")
        self.on_reset_button_clicked()
        update_available_specs()
        self._update_options()
        self.display_query_result()

        # setup automations
        self.w_update.on_click(self.on_update_button_clicked)
        self.w_reset.on_click(self.on_reset_button_clicked)
        self._set_specs_observers()
        self.w_select_sample_id.observe(handler=self.on_battery_id_change, names='value')
        self.w_validate.on_click(lambda arg: self.on_validate_button_clicked(validate_callback_f))


    @property
    def selected_sample_id(self):
        return self.w_select_sample_id.value

    @property
    def selected_sample_dict(self):
        return dict_to_formatted_json(
            query_available_samples(write_pd_query_from_dict({'battery_id': self.w_select_sample_id.value})).iloc[0])

    @property
    def current_specs(self):
        """
        A dictionary representing the current specs set by the user, that can be used in a query
        to filter the available samples.
        """
        return {
            'manufacturer': self.w_specs_manufacturer.value,
            'composition.description': self.w_specs_composition.value,
            'capacity.nominal': self.w_specs_capacity.value,
            'form_factor': self.w_specs_form_factor.value,
            # 'metadata.creation_datetime': self.w_specs_metadata_creation_date.value
        }

    def _build_sample_specs_options(self, spec_field):
        """
        Returns a `(option_string, battery_id)` list.
        The specs currently set are used to filter the sample list.
        The current `spec_field` is removed from the query, as we want to count how many samples correspond to each
        available value of the `spec_field`.
        """
        spec_field_options_list = query_available_specs(spec_field)

        # setup sample query filter from current specs and remove current field from query
        sample_query_filter_dict = self.current_specs.copy()
        sample_query_filter_dict[spec_field] = None

        # perform query of samples
        if self._DEBUG:
            print("\nSPEC FIELD: ", spec_field)
            print(f"       {spec_field_options_list}")
            print(" QUERY: ", sample_query_filter_dict)

        qres = query_available_samples(write_pd_query_from_dict(sample_query_filter_dict),
                                       project=[spec_field, 'battery_id']).sort_values('battery_id')

        # count values
        value_counts = qres[spec_field].value_counts()
        if self._DEBUG:
            print(' counts:', value_counts.to_dict())
        options_pairs = [(f"(no filter)  [{value_counts.sum()}]", None)]
        options_pairs.extend([(f"{value}  [{value_counts.get(value, 0)}]", value) for value in spec_field_options_list])
        return options_pairs

    def _build_sample_id_options(self):
        """Returns a (option_string, battery_id) list."""
        table = query_available_samples(write_pd_query_from_dict(self.current_specs)).sort_values('battery_id')
        return [("", None)] + [(f"<{row['battery_id']:5}>   \"{row['metadata.name']}\"", row['battery_id']) for index, row in table.iterrows()]
    
    def _update_options(self):
        # first save current values to preserve them
        w_specs_manufacturer_value = self.w_specs_manufacturer.value
        w_specs_composition_value = self.w_specs_composition.value
        w_specs_capacity_value = self.w_specs_capacity.value
        w_specs_form_factor_value = self.w_specs_form_factor.value
        self.w_specs_manufacturer.options = self._build_sample_specs_options('manufacturer')
        self.w_specs_manufacturer.value = w_specs_manufacturer_value
        self.w_specs_composition.options = self._build_sample_specs_options('composition.description')
        self.w_specs_composition.value = w_specs_composition_value
        self.w_specs_capacity.options = self._build_sample_specs_options('capacity.nominal')
        self.w_specs_capacity.value = w_specs_capacity_value
        self.w_specs_form_factor.options = self._build_sample_specs_options('form_factor')
        self.w_specs_form_factor.value = w_specs_form_factor_value
        self.w_select_sample_id.options = self._build_sample_id_options()
        self.w_select_sample_id.value = None

    def update_options(self):
        """Update the specs' options."""
        if self._DEBUG:
            print(f'updating options!')
        self._unset_specs_observers()
        self._update_options()
        self._set_specs_observers()

    def display_query_result(self):
        self.w_query_result.clear_output()
        with self.w_query_result:
            print(f'Query:\n  {self.current_specs}')
            display(query_available_samples(write_pd_query_from_dict(self.current_specs)))

    def display_sample_preview(self):
        self.w_sample_preview.clear_output()
        if self.w_select_sample_id.value is not None:
            with self.w_sample_preview:
                # display(query_available_samples(write_pd_query_from_dict({'battery_id': self.w_select_sample_id.value})))
                print(query_available_samples(write_pd_query_from_dict({'battery_id': self.w_select_sample_id.value})).iloc[0])

    def update_validate_button_state(self):
        self.w_validate.disabled = (self.w_select_sample_id.value is None)

    # def on_specs_value_change(self, which):
    #     def update_fields(_):
    #         self.display_query_result()
    #         self.update_options()#which)
    #     return update_fields

    def on_specs_value_change(self, _=None):
        self.update_options()
        self.display_query_result()

    def on_update_button_clicked(self, _=None):
        update_available_specs()
        update_available_samples()
        self.update_options()
        self.display_query_result()
        # notice: if the old value is not available anymore, an error might be raised!

    def on_reset_button_clicked(self, _=None):
        self.w_specs_manufacturer.value = None
        self.w_specs_composition.value = None
        self.w_specs_form_factor.value = None
        self.w_specs_capacity.value = None
        # self.w_specs_metadata_creation_date.value = None
        # self.w_specs_metadata_creation_process.value = None

    def on_battery_id_change(self, _=None):
        self.display_sample_preview()
        self.update_validate_button_state()

    def on_validate_button_clicked(self, callback_function):
        # call the validation callback function
        return callback_function(self)

    def _set_specs_observers(self):
        self.w_specs_manufacturer.observe(handler=self.on_specs_value_change, names='value')
        self.w_specs_composition.observe(handler=self.on_specs_value_change, names='value')
        self.w_specs_capacity.observe(handler=self.on_specs_value_change, names='value')
        self.w_specs_form_factor.observe(handler=self.on_specs_value_change, names='value')
        # self.w_specs_metadata_creation_date.observe(handler=self.update_options, names='value')
    
    def _unset_specs_observers(self):
        self.w_specs_manufacturer.unobserve(handler=self.on_specs_value_change, names='value')
        self.w_specs_composition.unobserve(handler=self.on_specs_value_change, names='value')
        self.w_specs_capacity.unobserve(handler=self.on_specs_value_change, names='value')
        self.w_specs_form_factor.unobserve(handler=self.on_specs_value_change, names='value')
        # self.w_specs_metadata_creation_date.unobserve(handler=self.update_options, names='value')

w_sample_from_specs = SampleFromSpecs(test_validate_sample_callback_function)
w_sample_from_specs

SampleFromSpecs(children=(GridBox(children=(Select(description='Manufacturer:', layout=Layout(height='100px', …

### **From Synthesis recipe**

In [9]:
class SampleFromRecipe(ipw.VBox):

    # BOX_LAYOUT_1 = {'width': '40%'}
    BOX_LAYOUT_2 = {'width': '100%'}
    BOX_STYLE = {'description_width': '25%'}
    BUTTON_STYLE = {'description_width': '30%'}
    BUTTON_LAYOUT = {'margin': '5px'}
    OUTPUT_LAYOUT = {'width': '100%', 'margin': '5px', 'padding': '5px', 'border': 'solid 2px'}  #'max_height': '500px'
    MAIN_LAYOUT = {'width': '100%', 'padding': '10px', 'border': 'solid blue 2px'}

    def __init__(self, validate_callback_f):

        # initialize widgets
        self.w_specs_label = ipw.HTML(value="<h2>Battery Specifications</h2>")
        self.w_specs_manufacturer = ipw.Dropdown(
            description="Manufacturer:",
            placeholder="Enter manufacturer",
            layout=self.BOX_LAYOUT_2, style=self.BOX_STYLE)
        self.w_specs_composition = ipw.Dropdown(
            description="Composition:",
            placeholder="Enter composition",
            layout=self.BOX_LAYOUT_2, style=self.BOX_STYLE)
        self.w_specs_capacity = ipw.Dropdown(
            description="Nominal capacity:",
            placeholder="Enter nominal capacity",
            layout=self.BOX_LAYOUT_2, style=self.BOX_STYLE)
        self.w_specs_form_factor = ipw.Dropdown(
            description="Form factor:",
            placeholder="Enter form factor",
            layout=self.BOX_LAYOUT_2, style=self.BOX_STYLE)

        self.w_recipe_label = ipw.HTML(value="<h2>Recipe Specifications</h2>")
        self.w_recipe_select = ipw.Select(
            rows=10, value=None,
            description="Select Recipe:",
            style=self.BOX_STYLE, layout=self.BOX_LAYOUT_2)
        self.w_recipe_preview = ipw.Output(
            layout=self.OUTPUT_LAYOUT)
        self.w_sample_metadata_name = ipw.Text(
            description="Sample name:",
            placeholder="Enter a name for this sample",
            layout=self.BOX_LAYOUT_2, style=self.BOX_STYLE)
        self.w_sample_metadata_creation_process = ipw.Textarea(
            description="Description:",
            placeholder="Describe the creation process",
            layout=self.BOX_LAYOUT_2, style=self.BOX_STYLE)

        self.w_update = ipw.Button(
            description="Update",
            button_style='', tooltip="Update available specs and recipies", icon='refresh',
            style=self.BUTTON_STYLE, layout=self.BUTTON_LAYOUT)
        self.w_reset = ipw.Button(
            description="Clear",
            button_style='danger', tooltip="Clear fields", icon='times',
            style=self.BUTTON_STYLE, layout=self.BUTTON_LAYOUT)

        self.w_validate = ipw.Button(
            description="Validate",
            button_style='success', tooltip="Validate the selected sample", icon='check',
            disabled=True,
            style=self.BUTTON_STYLE, layout=self.BUTTON_LAYOUT)
        # self.w_cookit = ipw.Button(
        #     description="Launch Recipy",
        #     button_style='info', tooltip="Cook it!", icon='blender',
        #     disabled=False,
        #     style=self.BUTTON_STYLE)

        # initialize widgets
        super().__init__()
        self.children = [
            ipw.VBox([
                self.w_specs_label,
                ipw.GridBox([
                    self.w_specs_manufacturer,
                    self.w_specs_composition,
                    self.w_specs_capacity,
                    self.w_specs_form_factor,
                ], layout=ipw.Layout(grid_template_columns="repeat(2, 45%)")),
                self.w_recipe_label,
                ipw.GridBox([
                    self.w_recipe_select,
                    self.w_recipe_preview,
                    self.w_sample_metadata_name,
                    self.w_sample_metadata_creation_process,
                ], layout=ipw.Layout(grid_template_columns="repeat(2, 45%)")),
                ipw.HBox([self.w_update, self.w_reset], layout={'justify_content': 'center', 'margin': '5px'}),
            ], layout=self.MAIN_LAYOUT),
            self.w_validate,
            # self.w_cookit,
        ]

        # initialize options
        self.on_update_button_clicked()
        self.display_recipe_preview()

        # setup automations
        self.w_update.on_click(self.on_update_button_clicked)
        self.w_reset.on_click(self.on_reset_button_clicked)
        self.w_recipe_select.observe(self.on_recipe_value_change, names='value')
        self.w_sample_metadata_name.observe(self.update_validate_button_state, names='value')
        self.w_validate.on_click(lambda arg: self.on_validate_button_clicked(validate_callback_f))


    @property
    def selected_specs_dict(self):
        return dict_to_formatted_json({
            'manufacturer': self.w_specs_manufacturer.value,
            'composition.description': self.w_specs_composition.value,
            'capacity.nominal': self.w_specs_capacity.value,
            'capacity.units': 'mAh',
            'form_factor': self.w_specs_form_factor.value,
        })
    
    @property
    def selected_recipe_dict(self):
        return dict_to_formatted_json({
            'recipe': self.w_recipe_select.value,
            'metadata.name': self.w_sample_metadata_name.value,
            'metadata.creation_process': self.w_sample_metadata_creation_process.value,
        })

    @staticmethod
    def _build_recipies_options():
        """Returns a (name, description) list."""
        dic = query_available_recipies()
        return [("", None)] + [(name, descr) for name, descr in dic.items()]

    def display_recipe_preview(self):
        self.w_recipe_preview.clear_output()
        with self.w_recipe_preview:
            if self.w_recipe_select.value:
                print(', '.join(self.w_recipe_select.value))

    def update_validate_button_state(self, _=None):
        self.w_validate.disabled = (self.w_recipe_select.value is None) or (len(self.w_sample_metadata_name.value) == 0)

    def on_recipe_value_change(self, _=None):
        self.update_validate_button_state()
        self.display_recipe_preview()

    def on_update_button_clicked(self, _=None):
        update_available_specs()
        update_available_recipies()
        self.w_specs_manufacturer.options = query_available_specs('manufacturer')
        self.w_specs_composition.options = query_available_specs('composition.description')
        self.w_specs_capacity.options = query_available_specs('capacity.nominal')
        self.w_specs_form_factor.options = query_available_specs('form_factor')
        self.w_recipe_select.options = self._build_recipies_options()

    def on_reset_button_clicked(self, _=None):
        self.w_specs_manufacturer.value = None
        self.w_specs_composition.value = None
        self.w_specs_form_factor.value = None
        self.w_specs_capacity.value = None
        self.w_recipe_select.value = None
        self.w_sample_metadata_name.value = ''
        self.w_sample_metadata_creation_process = ''

    def on_validate_button_clicked(self, callback_function):
        # call the validation callback function
        return callback_function(self)


w_sample_from_recipe = SampleFromRecipe(test_validate_specs_recipe_callback_function)
w_sample_from_recipe

SampleFromRecipe(children=(VBox(children=(HTML(value='<h2>Battery Specifications</h2>'), GridBox(children=(Dro…

### MAIN Sample Selection window

In [14]:
## works only with ipywidgets 8.0

# w_sample_input_stack = ipw.Stacked([w_sample_from_id, w_sample_from_specs, w_sample_from_recipe])
# ipw.jslink((w_sample_source, 'index'), (w_sample_input_stack, 'selected_index'))
# w_sample = ipw.VBox([w_sample_source, w_sample_input_stack])
# w_sample

In [77]:
class MainPanel(ipw.VBox):
    
    _SAMPLE_INPUT_LABELS = ['Select from ID', 'Select from Specs', 'Make from Recipe']
    _SAMPLE_INPUT_METHODS = ['id', 'specs', 'recipe']
    w_header = ipw.HTML(value="<h1>Aurora</h1>")

    def return_selected_sample(self, sample_widget_obj):
        self.selected_battery_sample = BatterySample.parse_obj(sample_widget_obj.selected_sample_dict)
        self.post_sample_selection()
    
    def return_selected_specs_recipe(self, sample_widget_obj):
        self.selected_battery_specs = BatterySpecs.parse_obj(sample_widget_obj.selected_specs_dict)
        self.selected_recipe = sample_widget_obj.selected_recipe_dict
    
    def post_sample_selection(self):
        print('POST')
        self.w_main_accordion.selected_index = 1
    
    def reset_sample_selection(self, _=None):
        self.selected_battery_sample = None
        self.selected_battery_specs = None
        self.selected_recipe = None
    
    @property
    def sample_selection_method(self):
        if self.w_sample_selection_tab.selected_index is not None:
            return self._SAMPLE_INPUT_METHODS[self.w_sample_selection_tab.selected_index]
    
    def __init__(self):
        
        # initialize variables
        self.reset_sample_selection()

        # Sample selection
        self.w_sample_from_id = SampleFromId(self.return_selected_sample)
        self.w_sample_from_specs = SampleFromSpecs(self.return_selected_sample)
        self.w_sample_from_recipe = SampleFromRecipe(self.return_selected_specs_recipe)

        self.w_sample_selection_tab = ipw.Tab(
            children=[self.w_sample_from_id, self.w_sample_from_specs, self.w_sample_from_recipe],
            selected_index=None)
        for i, title in enumerate(self._SAMPLE_INPUT_LABELS):
            self.w_sample_selection_tab.set_title(i, title)

        # Cycling
        self.w_test = ipw.Label('This is the Test part')
        
        # MAIN ACCORDION
        self.w_main_accordion = ipw.Accordion(children=[self.w_sample_selection_tab, self.w_test])
        self.w_main_accordion.set_title(0, 'Sample selection')
        self.w_main_accordion.set_title(1, 'Cycling Protocol')

        super().__init__()
        self.children = [
            self.w_header,
            self.w_main_accordion,
        ]
        
        # setup automations
        # reset selected sample/specs/recipe when the user selects another input tab
        self.w_sample_selection_tab.observe(self.reset_sample_selection, names='selected_index')

w_main = MainPanel()
w_main

MainPanel(children=(HTML(value='<h1>Aurora</h1>'), Accordion(children=(Tab(children=(SampleFromId(children=(Se…

In [99]:
w_main.sample_selection_method

'recipe'

In [100]:
w_main.selected_battery_sample

In [101]:
w_main.selected_battery_specs

BatterySpecs(manufacturer='BIG-MAP', composition=BatteryComposition(description='C | LP57 | NMC811', cathode='C', anode='NMC811', electrolyte='LP57'), form_factor='2032', capacity=BatteryCapacity(nominal=1.539, actual=None, units='mAh'))

In [102]:
w_main.selected_recipe

{'recipe': ['pomodoro',
  'mozzarella',
  'funghi',
  'zucchine',
  'melanzane',
  'peperoni'],
 'metadata': {'name': 'xdsg', 'creation_process': 'sdgsg'}}